In [ ]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
from google.colab import drive
!pip install torchmetrics                # ONLY FOR FIRST EXECUTION

# --- CONFIGURATIONS: SET PATHS, DEVICE, MODEL AND HYPERPARAMETERS ---

PATH_DRIVE = "/content/drive"
PATH_EXPORT = "/content/drive/MyDrive"
PATH_FILE = "/content/drive/MyDrive/SPair-71k.tar.gz"
PATH_DB = "/content/SPair-71k"
PATH_TEST = "/content/SPair-71k/PairAnnotation/test"
PATH_TRAIN = "/content/SPair-71k/PairAnnotation/trn"
ALL_PAIRS_PATH = "/content/SPair-71k/Layout/small/test.txt"
ALL_TRAIN_PAIRS_PATH = "/content/SPair-71k/Layout/small/trn.txt"

IMAGE_FOLDER_NAME = "/content/SPair-71k/JPEGImages"
PTH_PATH = "/content/drive/MyDrive/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth"

PATH_RES = "/content/Results"
os.makedirs(PATH_RES, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"          
MODEL_VERSION = "dinov2"         # "dinov2", "dinov3", "sam"
TUNING = True                     # TRAINING OR INFERENCE

IMAGE_SIZE = 224
PATCH_SIZE = 14 if MODEL_VERSION == "dinov2" else 16
H_PATCH, W_PATCH = IMAGE_SIZE // PATCH_SIZE, IMAGE_SIZE // PATCH_SIZE
ALPHA = [0.05, 0.1, 0.2]
CATEGORIES = ["aeroplane"]

# --- IF SAM MODEL ---

if MODEL_VERSION == "sam":
    !pip install git+https://github.com/facebookresearch/segment-anything.git 
    from segment_anything import sam_model_registry, SamPredictor        # DOWNLOAD
    PTH_PATH = "/content/drive/MyDrive/sam_vit_b_01ec64.pth"
    predictor = None

# --- DATASET CLASS ---

class SPair71kDataset(Dataset):
    def __init__(self, category_filter=None):
        self.pair_files = []
        self.path = ALL_TRAIN_PAIRS_PATH if TUNING else ALL_PAIRS_PATH
        self.image_path = PATH_TRAIN if TUNING else PATH_TEST
        file = open(self.path, "r")   # os.listdir(self.path)

        for line in file:
            (pair_id, category) = line.strip().split(".json")[0].split(":")
            if TUNING or category == category_filter:
                self.pair_files.append((line, category))

        file.close()        
        return

    def __len__(self):
        return len(self.pair_files)

    def __getitem__(self, idx):
        (json_file, category) = self.pair_files[idx]
        json_path = os.path.join(self.image_path, json_file.strip() + ".json")
        
        file = open(json_path, "r")
        annotation = json.load(file)
        file.close()

        src_name = annotation["src_imname"]
        trg_name = annotation["trg_imname"]
        src_path = os.path.join(IMAGE_FOLDER_NAME, category, src_name)
        trg_path = os.path.join(IMAGE_FOLDER_NAME, category, trg_name)

        return {
            "src_path": src_path,
            "trg_path": trg_path,
            "src_kps": np.array(annotation["src_kps"]),
            "trg_kps": np.array(annotation["trg_kps"]),
            "trg_bndbox": np.array(annotation["trg_bndbox"]),           # GET DATA
            "src_name": src_name,
            "trg_name": trg_name
        }

# --- IMAGE PREPROCESSING ---

preprocess = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- HELPER FUNCTIONS ---

# UNFREEZE ONLY THE LAST num_last_blocks LAYERS AND THE FINAL LAYER NORM

def setup_light_finetuning(model, num_last_blocks):
    blocks_to_unfreeze = model.blocks[-num_last_blocks:]
    N_Params = 0
    N_Free_Params = 0

    for param in model.parameters():                # FREEZE ALL
        N_Params += param.numel()
        param.requires_grad = False

    for block in blocks_to_unfreeze:

        for param in block.parameters():
            N_Free_Params += param.numel()           # UNFREEZE
            param.requires_grad = True

    for param in model.norm.parameters():          # UNFREEZE NORM
        N_Free_Params += param.numel()
        param.requires_grad = True

    # NUMBERS    

    print("Total parameters:", N_Params)
    print("Total trainable:", N_Free_Params)
    print("Percentage trainable:", round(100 * N_Free_Params / N_Params, 2), "%")
    return model

# --- LOAD AND PREPROCESS IMAGE. EXTRACT PATCH-WISE FEATURES FROM MODEL. --- 
# --- RESHAPE AND NORMALIZE FEATURE MAPS. --- 
# --- FINALLY, RETURN THE EXTRACTED FEATURE MAP AND THE ORIGINAL DIMENSIONS. --    

def get_descriptors(img_path, grad):
    img = Image.open(img_path).convert("RGB")
    (w, h) = img.size
    input_tensor = preprocess(img).unsqueeze(0).to(DEVICE)

    with torch.set_grad_enabled(grad):

        if MODEL_VERSION == "dinov2":
            feats = model.get_intermediate_layers(input_tensor, n=1)[0]
            feats = feats.reshape(1, H_PATCH, W_PATCH, feats.shape[2])

        elif MODEL_VERSION == "dinov3":
            x = model.forward_features(input_tensor)["x_norm_patchtokens"]
            feats = x.reshape(1, H_PATCH, W_PATCH, x.shape[-1])

        elif MODEL_VERSION == "sam":
            predictor.set_image(np.array(img))
            feats = predictor.get_image_embedding()
            feats = feats.permute(0, 2, 3, 1)

    feats = F.normalize(feats, dim=-1)
    return (feats, w, h)

# --- MATCH KEYPOINTS. ---
# --- EXTRACT FEATURE SIZE, CREATE PROBABILITY GRID FOR SOFT ARG MAX

def match_keypoints(batch, grad, tau=0.05):
    src_kps = batch["src_kps"][0].numpy()
  
    (feat_src, sw, sh) = get_descriptors(batch["src_path"][0], grad)          # GET DESCRIPTORS
    (feat_trg, tw, th) = get_descriptors(batch["trg_path"][0] , grad)

    (_, Hf, Wf, D) = feat_trg.shape
    trg_flat = feat_trg[0].reshape(Hf * Wf, D)
    pred_kps = []

    (ys, xs) = torch.meshgrid(
                  torch.arange(Hf, device=DEVICE),
                  torch.arange(Wf, device=DEVICE),
                  indexing="ij")

    for i in range(src_kps.shape[0]):
        sx = int(src_kps[i, 0] * Wf / sw)
        sy = int(src_kps[i, 1] * Hf / sh)
        sx = torch.clamp(torch.tensor(sx, device=DEVICE), 0, Wf - 1)
        sy = torch.clamp(torch.tensor(sy, device=DEVICE), 0, Hf - 1)
        src_desc = feat_src[0, sy, sx, :]

        sim = torch.matmul(trg_flat, src_desc)            # COSINE SIMILARITY

        if TUNING:
            coords = torch.stack([xs.flatten(), ys.flatten()], dim=1).float()

            prob = F.softmax(sim / tau, dim=0)
            pred_patch = (coords * prob[:, None]).sum(dim=0)
            (px, py) = (pred_patch[0], pred_patch[1])

        else:
            best_idx = sim.argmax()
            px = best_idx % Wf                      # ONLY INFERENCE
            py = best_idx // Wf
        
        pred_x = (px + 0.5) * (tw / Wf)
        pred_y = (py + 0.5) * (th / Hf)
        pred_kps.append(torch.stack([pred_x, pred_y]))

    return torch.stack(pred_kps)

# --- VISUALIZE RESULTS AND COMPARE CORRECT AND PREDICTED KEYPOINTS ON TARGET IMAGE. ---    

def visualize_keypoints(src_path, trg_path, src_kps, pred_kps, trg_kps):
    src_img = cv2.imread(src_path)[:, :, ::-1]
    trg_img = cv2.imread(trg_path)[:, :, ::-1]

    (_, axes) = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(src_img)
    axes[0].scatter(src_kps[:,0], src_kps[:,1], c="r", s=40, label="src_kps")
    axes[0].set_title("Source Image")

    axes[1].imshow(trg_img)
    axes[1].scatter(pred_kps[:,0], pred_kps[:,1], c="b", s=40, label="pred_kps")
    axes[1].scatter(trg_kps[:,0], trg_kps[:,1], c="g", s=40, marker="X", label="gt_kps")
    axes[1].set_title("Target Image")

    plt.legend()
    plt.show()

# --- TRAINING FUNCTION. ---

def Train(model, number_of_Epochs, learning_rate, layers, tau=0.05):
    global_loss = float('inf')
    criterion = torch.nn.SmoothL1Loss()
    dataset = SPair71kDataset()
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)           # INITIALIZATION
    params = []

    print()
    print("Trying with", layers, "free layers")
    model = setup_light_finetuning(model, layers)

    for p in model.parameters():
        if p.requires_grad:                  # TRAINABLE PARAMETERS
            params.append(p)

    optimizer = torch.optim.Adam(params, lr=learning_rate)

    for epoch in range(number_of_Epochs):
        print()
        print("Starting epoch", epoch)
        pbar = tqdm(dataloader, desc="Training")

        for batch in pbar:
            trg_kps = torch.tensor(batch["trg_kps"][0], device=DEVICE, dtype=torch.float32)

            pred_kps = match_keypoints(batch, grad=True, tau=tau)     # PREDICTION WITH SOFTMAX

            loss = criterion(pred_kps, trg_kps)           # LOSS

            optimizer.zero_grad()           # UPDATING
            loss.backward()
            optimizer.step()

            # UPDATING BAR AND GLOBAL LOSS

            pbar.set_postfix(loss=loss.item())

        if loss < global_loss:
            path = os.path.join(PATH_EXPORT, "best_model.pth")          # UPDATE ON DRIVE
            torch.save(model.state_dict(), path)
            global_loss = loss

    print("Training finished")
    return

# --- EVALUATION FUNCTION ---
# --- CONSIDER ONLY ONE CATEGORY. --- 
# --- FOR EACH PAIR: LOAD SOURCE AND TARGET IMAGES ---
# --- EXTRACT DESCRIPTORS AND CORRECT KEYPOINTS AND RESCALE COORDINATES --- 
# --- FOR EACH ALPHA: USE COSINE SIMILARITY IN ORDER TO FIND THE CORRESPONDING POINT IN THE TARGET IMAGE. --- 
# --- FINALLY, RETURN THE CORRECT GENERATED KEYPOINTS (USING THEIR DISTANCE FROM THE ORIGINAL ONE) RATIO. ---
# --- OPTIONALLY: VISUALIZE RESULTS. ---

def run_evaluation(loader, category, visualize=False):
    total_correct = {alpha: 0 for alpha in ALPHA}
    total_points = 0
    pbar = tqdm(loader, desc="Evaluating " + category)

    for batch in pbar:
        src_kps = batch["src_kps"][0].numpy()
        trg_kps = batch["trg_kps"][0].numpy()

        pred_kps = match_keypoints(batch, False).numpy()

        max_dim = max(batch["trg_bndbox"][0][2]-batch["trg_bndbox"][0][0], batch["trg_bndbox"][0][3]-batch["trg_bndbox"][0][1])
        total_correct_image = {alpha: 0 for alpha in ALPHA}
        total_points_image = 0                                      # FOR THE CURRENT IMAGE

        for i in range(len(src_kps)):
            dist = np.linalg.norm(pred_kps[i] - trg_kps[i])
            total_points += 1
            total_points_image += 1

            for alpha in ALPHA:
                if dist <= alpha * max_dim:                     # PREDICTION IS CORRECT?
                    total_correct[alpha] += 1
                    total_correct_image[alpha] += 1

        # PRINT PER IMAGE

        print()
        print("Results for (", batch["src_name"][0], ",", batch["trg_name"][0], ")")

        for alpha in ALPHA:
            total_correct_image[alpha] = round(100 * total_correct_image[alpha] / total_points_image, 2)
            print("PCK@", str(alpha), ": ", str(total_correct_image[alpha]), "%")

        mean_pck_image = round(sum(total_correct_image.values()) / len(ALPHA), 2)
        print("MEAN PCK:", mean_pck_image, "%")
        total_correct_image["MEAN"] = mean_pck_image

        # SAVE JSON

        path = os.path.join(PATH_RES, 
                            "pck_results_for_" + MODEL_VERSION + "_" + batch["src_name"][0] + "_" + batch["trg_name"][0] + ".json")
        file = open(path, "w")
        json.dump(total_correct_image, file, indent=4)
        file.close()

        # VISUALIZE

        if visualize:
            visualize_keypoints(batch["src_path"][0], batch["trg_path"][0], src_kps, pred_kps, trg_kps)

    # CATEGORY PCK

    print()
    print("="*40)
    print("Category:", category)

    for alpha in ALPHA:
        total_correct[alpha] = round(100 * total_correct[alpha] / total_points, 2)
        print("PCK@", str(alpha), ": ", str(total_correct[alpha]), "%")

    mean_pck = round(sum(total_correct.values()) / len(ALPHA), 2)
    total_correct["MEAN"] = mean_pck
    print("MEAN PCK:", mean_pck, "%")

    path = os.path.join(PATH_RES, "pck_results_for_" + MODEL_VERSION + "_" + category + ".json")
    file = open(path, "w")
    json.dump(total_correct, file, indent=4)
    file.close()
    return

# --- MOUNT DRIVE AND EXTRACT DATASET ---

drive.mount(PATH_DRIVE, force_remount=True)
!tar -xzf {PATH_FILE}

# --- LOAD MODEL ---

print()
print("Loading ", MODEL_VERSION, " model...")

if MODEL_VERSION == "dinov2":
    model = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14_reg", pretrained=True).to(DEVICE)
elif MODEL_VERSION == "dinov3":
    model = torch.hub.load("facebookresearch/dinov3", "dinov3_vitb16", weights=PTH_PATH).to(DEVICE)
elif MODEL_VERSION == "sam":
    model = sam_model_registry["vit_b"](checkpoint=PTH_PATH).to(DEVICE)
    predictor = SamPredictor(model)

# --- TRAINING IF ENABLED ---

if TUNING:
    model.train()
    Train(model, number_of_Epochs=5, learning_rate=1e-5, layers=3)

model.eval()

# --- EVALUATION PER CATEGORY ---

for cat in CATEGORIES:
    print()
    print("Evaluating category: ", cat)

    dataset_cat = SPair71kDataset(category_filter=cat)
    loader = DataLoader(dataset_cat, batch_size=1, shuffle=False)
    run_evaluation(loader, cat, visualize=False)
